In [1]:
import pandas as pd

def get_dics_from_df(in_df):
    res_dics = in_df.to_dict('records')
    return res_dics

def get_df_from_dics(in_dics):
    res_df = pd.DataFrame.from_dict(dics)
    return res_df

def get_headers(in_df):
    res_df = list(in_df.columns.values)
    return res_df

def get_values(in_df, key):
    out_df = in_df[key].tolist()
    return out_df

def get_count(in_df):
    res_count = in_df.shape[0]
    return res_count

def get_distinct_count(in_df, keys):
    in_df['_concat_keys'] = ''
    for key in keys:
        in_df['_concat_keys'] += in_df[key].map(str)
    concats = in_df['_concat_keys'].to_list()
    concats = list(set(concats))
    concats_count = len(concats)
    return concats_count

def get_both(left_df, right_df, keys):
    res_df = pd.merge(left_df, right_df, how='outer', on=keys, indicator=True)
    res_df = res_df.loc[res_df['_merge'] == 'both']
    res_df = res_df.drop(columns=['_merge'])
    return res_df

def get_left(left_df, right_df, keys):
    res_df = pd.merge(left_df, right_df, how='outer', on=keys, indicator=True)
    res_df = res_df.loc[(res_df['_merge'] == 'left_only') | (res_df['_merge'] == 'both')]
    res_df = res_df.drop(columns=['_merge'])
    return res_df   

def get_left_only(left_df, right_df, keys):
    res_df = pd.merge(left_df, right_df, how='outer', on=keys, indicator=True)
    res_df = res_df.loc[res_df['_merge'] == 'left_only']
    res_df = res_df.drop(columns=['_merge'])
    return res_df    

def get_right(left_df, right_df, keys):
    res_df = pd.merge(left_df, right_df, how='outer', on=keys, indicator=True)
    res_df = res_df.loc[(res_df['_merge'] == 'right_only') | (res_df['_merge'] == 'both')]
    res_df = res_df.drop(columns=['_merge'])

def get_right_only(left_df, right_df, keys):
    res_df = pd.merge(left_df, right_df, how='outer', on=keys, indicator=True)
    res_df = res_df.loc[res_df['_merge'] == 'right_only']
    res_df = res_df.drop(columns=['_merge'])
    return res_df    

def get_join_on_left(left_df, right_df, keys):
    res_df = get_left(left_df, right_df, keys)
    return res_df

def get_join_on_left_break(left_df, right_df, keys):
    pass_df = get_both(left_df, right_df, keys)
    fail_df = get_left_only(left_df, right_df, keys)
    return pass_df, fail_df

def get_melt(in_df, key_cols, var_cols):
    out_df = in_df.melt(id_vars=key_cols, value_vars=var_cols, var_name='var', value_name='value')
    return out_df

def get_unmelt(in_df, keys, var_col, value_col):
    aux1_df = in_df
    aux1_df['_concat_keys'] = ''
    for key in keys:
        aux1_df['_concat_keys'] += aux1_df[key].map(str)
    aux2_df = aux1_df
    aux1_df = aux1_df.drop(columns=[var_col, value_col]).drop_duplicates().reset_index(drop=True)
    aux2_df = aux2_df[['_concat_keys', var_col, value_col]]
    aux2_df = aux2_df.pivot_table(index='_concat_keys', columns=var_col)[value_col]
    res_df = pd.merge(aux1_df, aux2_df, how='outer', on='_concat_keys', indicator=True)
    res_df = res_df.loc[(res_df['_merge'] == 'left_only') | (res_df['_merge'] == 'both')]
    res_df = res_df.drop(columns=['_merge'])
    res_df = res_df.drop(columns=['_concat_keys'])
    return res_df

def get_split_apply_combine(in_df, split_keys, apply_func):
    in_df['_concat_keys'] = ''
    for key in split_keys:
        in_df['_concat_keys'] += in_df[key].map(str)
    concats = in_df['_concat_keys'].to_list()
    concats = list(set(concats))
    temp_dics = []
    out_dics = []
    out_df = None
    for concat in concats:
        temp_df = in_df.loc[in_df['_concat_keys'] == concat]
        temp_df = temp_df.drop(columns=['_concat_keys'])    
        temp_dics = temp_df.to_dict('records')
        temp_dics = apply_func(temp_dics)
        out_dics += temp_dics
        out_df = pd.DataFrame.from_dict(out_dics)
    out_df = out_df.sort_values(by=split_keys).reset_index(drop=True)
    return out_df

def apply_example(dics):
    i = 0
    for dic in dics:
        if(dic['key'] == 'K0'):
            dic['B'] = -1
    return dics


In [ ]:

from pywinauto import application, Desktop

import time
import pyautogui

pyautogui.FAILSAFE = False

def _get_all_titles():
    titles = []
    windows = pyautogui.getAllWindows()
    for window in windows:
        titles.append(window.title)
    return titles    

def _get_all_windows():
    windows = pyautogui.getAllWindows()
    return windows

def _count_targets_from_str(target_str):
    count = 0
    titles = _get_all_titles()
    for title in titles:
        if(target_str in title):
            count += 1
    return count

def _get_targets_from_str(target_str):
    idxs = []
    titles = _get_all_titles()
    for idx, title in enumerate(titles):
        if(target_str in title):
            idxs.append(idx)
    windows = []
    for idx in idxs:
        windows.append(_get_all_windows()[idx])
    if(len(windows) > 0):
        return windows
    else:
        return None


def maximize_all():
    windows = _get_all_windows()
    for window in windows:
        window.maximize()
    return True

def minimize_all():
    windows = _get_all_windows()
    for window in windows:
        window.minimize()
    return True

def close_all():
    windows = _get_all_windows()
    for window in windows:
        window.close()
    return True

def maximize_target(target_str):
    windows = _get_targets_from_str(target_str)
    if(windows is not None):
        for window in windows:
            window.maximize()
        return True
    else:
        return False

def minimize_target(target_str):
    windows = _get_targets_from_str(target_str)
    if(windows is not None):
        for window in windows:
            window.minimize()   
        return True
    else:
        return False

def close_target(target_str):
    windows = _get_targets_from_str(target_str)
    if(windows is not None):
        for window in windows:
            window.close()
        return True
    else:
        return False

def restore_target(target_str):
    windows = _get_targets_from_str(target_str)
    if(windows is not None):
        for window in windows:
            window.restore()
        return True
    else:
        return False


def double_click_on_target(target_image_path):
    if(pyautogui.locateCenterOnScreen(target_image_path) is not None):
        x, y = pyautogui.locateCenterOnScreen(target_image_path)
        pyautogui.click(x, y, clicks=2, interval=0.25)
        return True
    else:
        return False    

def single_click_on_target(target_image_path):
    if(pyautogui.locateCenterOnScreen(target_image_path) is not None):
        x, y = pyautogui.locateCenterOnScreen(target_image_path)              #('./1.png')
        pyautogui.click(x, y, clicks=1, interval=0.25)
        return True
    else:
        return False    